In [38]:
# Instalar el cliente de Redis
!pip install redis

In [39]:
import redis

# Conexión a Redis local
r = redis.Redis(host='localhost', port=6380, decode_responses=True)
# Decodifica automáticamente las respuestas de Redis para que no devuelva siempre bytes.

# Probar la conexión
r.ping()

True

## 1

Comenzamos cargando personas (de la entidad Persona) como json asociados a una clave interna

In [40]:
import json

lidia = {
    "nombre": "Lidia",
    "apellido": "Padilla",
    "segundo_nombre": "NULL",
    "segundo_apellido": "Paredes",
    "tipo_documento": "LC",
    "numero_documento": 30690283,
    "sexo": "X",
    "fecha_nacimiento": "1997-03-07",
    "id_provincia": 1,
    "cobertura_medica": "Mixta"
}

r.set("persona:1000", json.dumps(lidia)) # Guarda la persona como un JSON en Redis

# Recuperacion del JSON desde Redis y conversion a un diccionacion de Python.
lidia_recuperada = json.loads(r.get("persona:1000"))
print(lidia_recuperada)

{'nombre': 'Lidia', 'apellido': 'Padilla', 'segundo_nombre': 'NULL', 'segundo_apellido': 'Paredes', 'tipo_documento': 'LC', 'numero_documento': 30690283, 'sexo': 'X', 'fecha_nacimiento': '1997-03-07', 'id_provincia': 1, 'cobertura_medica': 'Mixta'}


Editamos info de Lidia (porque por ejemplo se cambió de tipo de cobertura médica)

In [41]:
lidia_recuperada['cobertura_medica'] = 'CPE'
r.set("persona:1000", json.dumps(lidia_recuperada))  # Actualiza el JSON en Redis

# Recuperar la persona actualizada
lidia_actualizada = json.loads(r.get("persona:1000"))
print(lidia_actualizada)

{'nombre': 'Lidia', 'apellido': 'Padilla', 'segundo_nombre': 'NULL', 'segundo_apellido': 'Paredes', 'tipo_documento': 'LC', 'numero_documento': 30690283, 'sexo': 'X', 'fecha_nacimiento': '1997-03-07', 'id_provincia': 1, 'cobertura_medica': 'CPE'}


Ahora, cargamos a varias personas desde un archivo JSON (generado de nuestra tabla Persona) a un hash de Personas

In [42]:
import requests

url = "https://raw.githubusercontent.com/Dafydd8/TP2-TDVII/main/data/persona.json"

response = requests.get(url)
response.raise_for_status()

# Parsear el contenido como JSON
personas = response.json()

print(type(personas))
for persona in personas[:5]: # Imprimir las primeras 5 personas
    print(persona)


<class 'list'>
{'nombre': 'Lidia', 'apellido': 'Padilla', 'segundo_nombre': 'NULL', 'segundo_apellido': 'Paredes', 'tipo_documento': 'LC', 'numero_documento': 30690283, 'sexo': 'X', 'fecha_nacimiento': '1997-03-07', 'id_provincia': 1, 'cobertura_medica': 'CPE'}
{'nombre': 'Susana', 'apellido': 'Granados', 'segundo_nombre': 'NULL', 'segundo_apellido': 'NULL', 'tipo_documento': 'LC', 'numero_documento': 10200509, 'sexo': 'M', 'fecha_nacimiento': '1954-07-11', 'id_provincia': 1, 'cobertura_medica': 'CPE'}
{'nombre': 'Rufino', 'apellido': 'Coello', 'segundo_nombre': 'NULL', 'segundo_apellido': 'NULL', 'tipo_documento': 'DNI', 'numero_documento': 35688920, 'sexo': 'M', 'fecha_nacimiento': '1983-08-10', 'id_provincia': 1, 'cobertura_medica': 'Mixta'}
{'nombre': 'Pascuala', 'apellido': 'Cabo', 'segundo_nombre': 'NULL', 'segundo_apellido': 'NULL', 'tipo_documento': 'LE', 'numero_documento': 17946521, 'sexo': 'X', 'fecha_nacimiento': '2001-05-09', 'id_provincia': 1, 'cobertura_medica': 'CPE'}
{

In [43]:
i = 0
for persona in personas:
   s = "persona:100"+str(i)
   i += 1
   r.hset("personas", s, json.dumps(persona)) # personas es el "titulo" del hash/diccionario

Extraemos dos personas para chequear que están bien cargadas

In [44]:
persona1 = json.loads(r.hget("personas", "persona:1000"))
persona2 = json.loads(r.hget("personas", "persona:1001"))

print("Persona 1:", persona1)
print("Persona 2:", persona2)

Persona 1: {'nombre': 'Lidia', 'apellido': 'Padilla', 'segundo_nombre': 'NULL', 'segundo_apellido': 'Paredes', 'tipo_documento': 'LC', 'numero_documento': 30690283, 'sexo': 'X', 'fecha_nacimiento': '1997-03-07', 'id_provincia': 1, 'cobertura_medica': 'CPE'}
Persona 2: {'nombre': 'Susana', 'apellido': 'Granados', 'segundo_nombre': 'NULL', 'segundo_apellido': 'NULL', 'tipo_documento': 'LC', 'numero_documento': 10200509, 'sexo': 'M', 'fecha_nacimiento': '1954-07-11', 'id_provincia': 1, 'cobertura_medica': 'CPE'}


Modificamos un atributo de una de las personas (id_provincia), y lo seteamos en el hash.

In [45]:
persona1["id_provincia"] = 2
print(persona1)

{'nombre': 'Lidia', 'apellido': 'Padilla', 'segundo_nombre': 'NULL', 'segundo_apellido': 'Paredes', 'tipo_documento': 'LC', 'numero_documento': 30690283, 'sexo': 'X', 'fecha_nacimiento': '1997-03-07', 'id_provincia': 2, 'cobertura_medica': 'CPE'}


In [46]:
r.hset("personas", "persona:1000", json.dumps(persona1))

0

Chequeamos que se haya modificado correctamente

In [47]:
p1 = json.loads(r.hget("personas", "persona:1000"))

print("Persona 1:", p1)

Persona 1: {'nombre': 'Lidia', 'apellido': 'Padilla', 'segundo_nombre': 'NULL', 'segundo_apellido': 'Paredes', 'tipo_documento': 'LC', 'numero_documento': 30690283, 'sexo': 'X', 'fecha_nacimiento': '1997-03-07', 'id_provincia': 2, 'cobertura_medica': 'CPE'}


Eliminamos una persona del hash

In [48]:
r.hdel("personas", "persona:1000")  # Eliminar la persona 1

1

Vemos que ya no devuelve nada al intentar extraerla...

In [49]:
r.hget("personas", "persona:1000")

...y que no está en el hash.

In [37]:
personas_retrieved = r.hgetall("personas")
sorted_personas_retrieved = sorted(personas_retrieved.items(), key=lambda x: int(x[0].split(':')[1]))

# Mostrar ordenado
for key, value in sorted_personas_retrieved:
    print(f"{key}: {value}")

persona:1001: {"nombre": "Susana", "apellido": "Granados", "segundo_nombre": "NULL", "segundo_apellido": "NULL", "tipo_documento": "LC", "numero_documento": 10200509, "sexo": "M", "fecha_nacimiento": "1954-07-11", "id_provincia": 1, "cobertura_medica": "CPE"}
persona:1002: {"nombre": "Rufino", "apellido": "Coello", "segundo_nombre": "NULL", "segundo_apellido": "NULL", "tipo_documento": "DNI", "numero_documento": 35688920, "sexo": "M", "fecha_nacimiento": "1983-08-10", "id_provincia": 1, "cobertura_medica": "Mixta"}
persona:1003: {"nombre": "Pascuala", "apellido": "Cabo", "segundo_nombre": "NULL", "segundo_apellido": "NULL", "tipo_documento": "LE", "numero_documento": 17946521, "sexo": "X", "fecha_nacimiento": "2001-05-09", "id_provincia": 1, "cobertura_medica": "CPE"}
persona:1004: {"nombre": "Basilio", "apellido": "Mateo", "segundo_nombre": "Celestina", "segundo_apellido": "Pedrosa", "tipo_documento": "DNI", "numero_documento": 31535749, "sexo": "F", "fecha_nacimiento": "2012-09-27", 

In [15]:
new_person = {"nombre": "Juan", "apellido": "Castore", "segundo_nombre": "Ignacio", "segundo_apellido": "Bergioli", "tipo_documento": "DNI", "numero_documento": 42017743, "sexo": "M", "fecha_nacimiento": "1978-07-12", "id_provincia": 4, "cobertura_medica": "CPE"}
r.hset("personas", "persona:1002", json.dumps(new_person))
print("Persona 1002:", json.loads(r.hget("personas", "persona:1002")))

Persona 1002: {'nombre': 'Juan', 'apellido': 'Castore', 'segundo_nombre': 'Ignacio', 'segundo_apellido': 'Bergioli', 'tipo_documento': 'DNI', 'numero_documento': 42017743, 'sexo': 'M', 'fecha_nacimiento': '1978-07-12', 'id_provincia': 4, 'cobertura_medica': 'CPE'}


## 2

#### Lista de HCEs que no gestionan recetas digitales:
Nuestra "lista de tareas" contiene los HCE que no generan receta digitales, representando los que quedan por adaptar y modernizar a esta tecnología.

In [17]:
import pandas as pd

path = 'https://raw.githubusercontent.com/Dafydd8/TP2-TDVII/main/data/hce.csv'
df_hce = pd.read_csv(path)

hce_no_gestiona = df_hce[df_hce['generacion_receta_digital'] == 'No genera']['nombre'].tolist()

r.delete('hce_no_genera_recetas')
r.rpush('hce_no_genera_recetas', *hce_no_gestiona)

21

In [18]:
#recuperar la lista de hces que no generan recetas digitales (puede ser util, por ejemplo, para un area de modernizacion)
print(r.lrange('hce_no_genera_recetas', 0, -1))

['HCE Parra', 'HCE Blanco', 'HCE Royo', 'HCE Catalán', 'HCE Arana', 'HCE Almeida', 'HCE Montesinos', 'HCE Rocamora', 'HCE Villena', 'HCE Chaparro', 'HCE Alegria', 'HCE Nicolás', 'HCE Sebastián', 'HCE Esparza', 'HCE Morata', 'HCE Vilar', 'HCE Alcázar', 'HCE Falcó', 'HCE Jover', 'HCE Lobato', 'HCE Ferrándiz']


In [19]:
#eliminar una determinada HCE (por ejemplo, si se moderniza)
r.lrem('hce_no_genera_recetas', 1, 'HCE Royo')  # Elimina una ocurrencia de 'Hospital Italiano' (la única en este caso)
print(r.lrange('hce_no_genera_recetas', 0, -1))  # Verifica que se haya eliminado

['HCE Parra', 'HCE Blanco', 'HCE Catalán', 'HCE Arana', 'HCE Almeida', 'HCE Montesinos', 'HCE Rocamora', 'HCE Villena', 'HCE Chaparro', 'HCE Alegria', 'HCE Nicolás', 'HCE Sebastián', 'HCE Esparza', 'HCE Morata', 'HCE Vilar', 'HCE Alcázar', 'HCE Falcó', 'HCE Jover', 'HCE Lobato', 'HCE Ferrándiz']


## 3

Uso de claves temporales para detectar cuando un problema lleva mucho tiempo sin ser atendido

In [67]:
problema1 = {
    "cod_problema":1,
    "cod_refes":11,
    "tipo":"Infraestructura",
    "descripcion":"Techo con filtraciones en sala de espera"
}

problema2 = {
    "cod_problema":11,
    "cod_refes":1,
    "tipo":"Infraestructura",
    "descripcion":"Fallas eléctricas en quirófano"
}

problema3 = {
    "cod_problema":11,
    "cod_refes":1,
    "tipo":"Infraestructura",
    "descripcion":"INCENDIO!"
}

#r.setex("problema:100", 86400, json.dumps(problema1))  # Guarda el problema con una caducidad de 24 horas
#r.setex("problema:101", 7200, json.dumps(problema2))  # Guarda el problema con una caducidad de 2 horas (es más urgente)
r.setex("problema:102", 10, json.dumps(problema3))  # Guarda el problema con una caducidad de 30 segundos (es muy urgente) (y nos sirve para testear :) )

True

In [ ]:
# Imprimir un mensje por pantalla cuando a algún problema le quede menos de 1 hora de vida (se lo pasa a considerar urgente)
# y cuando expire

#problemas = [("problema:100", True), ("problema:101", True), ("problema:102", True)]  # Lista de problemas con su estado de mostrar mensaje
problemas = [("problema:102", True)] # Un solo problema para probar el código
while (len(problemas) > 0):
    for problema in problemas:
        cod = problema[0]
        mostrar = problema[1]
        ttl = r.ttl(cod)  # Obtiene el tiempo de vida restante de cada problema
        if mostrar and ttl <= 3600 :
            print("Queda menos de 1 hora para resolver el problema:", json.loads(r.get(cod)))
            problemas[problemas.index(problema)] = (cod, False) # Actualiza el estado para no mostrar el mensaje nuevamente
        elif ttl == 0:
            print("Problema expirado! Info:", json.loads(r.get(cod)))
            problemas.pop(problemas.index(problema))

Queda menos de 1 hora para resolver el problema: {'cod_problema': 11, 'cod_refes': 1, 'tipo': 'Infraestructura', 'descripcion': 'INCENDIO!'}
Problema expirado! Info: {'cod_problema': 11, 'cod_refes': 1, 'tipo': 'Infraestructura', 'descripcion': 'INCENDIO!'}
[]
